In [12]:
import numpy as np

def waven(T, h):
    """
    Calculate the wavenumber k for gravity waves using Newton's method.

    Parameters:
    T : float or np.ndarray
        Wave period [s]. Can be a scalar or an array.
    h : float
        Water depth [m].

    Returns:
    k : float or np.ndarray
        Wavenumber [1/m]. Matches the shape of T.
    """

    T = np.atleast_1d(T)  # Ensure T is an array
    jmax = 20
    xacc = 0.0001
    g = 9.80665
    w = 2 * np.pi / T
    w2h = w**2 * h / g

    # Initial guess for kh
    kh = w2h.copy()

    for _ in range(jmax):
        tx = np.tanh(kh)
        f = kh * tx - w2h
        df = kh * (1 - tx**2) + tx
        dx = f / df
        kh -= dx

        if np.all(np.abs(dx) < xacc):
            break
    else:
        raise ValueError("waven exceeded maximum iterations")

    k = kh / h
    return k




def ub_f(T, H, h):
    """
    Calculate near-bottom wave-orbital velocity amplitude.

    Parameters:
    T : float or np.ndarray
        Significant wave period. Can be a scalar or an array.
    H : float or np.ndarray
        Significant wave height (2 * amplitude of surface wave). Matches the shape of T.
    h : float
        Water depth.

    Returns:
    ub : float or np.ndarray
        Near-bottom wave-orbital velocity amplitude. Matches the shape of T and H.
    """

    T = np.atleast_1d(T)
    H = np.atleast_1d(H)
    
    if T.shape != H.shape:
        raise ValueError("T and H must have the same shape")

    # Get the wave number using the waven function
    k = waven(T, h)

    # Calculate near-bottom wave-orbital velocity amplitude
    twopi = 2 * np.pi
    w = twopi / T
    kh = k * h
    amp = H / (2.0 * np.sinh(kh))
    ub = w * amp

    return ub




def uRMS_ubr_Soulsby(Hs, Tp, h):
    """
    Calculate orbital velocity under JONSWAP spectra using Soulsby approximation.

    Parameters:
    Hs : float or np.ndarray
        Significant wave height (m). Can be a scalar or an array.
    Tp : float or np.ndarray
        Wave period (s). Can be a scalar or an array.
    h : float or np.ndarray
        Water depth (m). Can be a scalar or an array.

    Returns:
    Ubr : float or np.ndarray
        Monochromatic representative wave velocity (m/s). Arrays if inputs are arrays.
    
    Reference:
    Soulsby (2006), "Simplified calculation of wave orbital velocities",
    Report TR 155 Release 1.0 Feb 2006. HR Wallingford.
    Section 3.2, equation 28.

    Note:
    1.28*Tz ~= Tp
    sqrt(2)*uRMS = ubr (monochromatic wave with same variance)
    """
    
    Tz = Tp / 1.28
    g = 9.81

    # Vectorized operations using numpy
    term = - ((3.65 / Tz) * (np.sqrt(h / g)))**2.1
    uRMS = (Hs / 4.) * np.sqrt(g / h) * np.exp(term)
    ubr = np.sqrt(2) * uRMS

    return ubr

In [13]:
# Example usage:
T = 12  # wave periods in seconds
H = 4   # wave heights in meters
h = 50  # water depth in meters

ubr_lwt = ub_f(T, H, h)
ubr_slb = uRMS_ubr_Soulsby(H, T, h)

print('Near-bottom wave-orbital velocity amplitude (Ubr)')
print(f"LINEAR WAVE THEORY :    {ubr_lwt[0]} m/s")
print(f"SOULSBY APPROXIMATION:  {ubr_slb} m/s")

Near-bottom wave-orbital velocity amplitude (Ubr)
LINEAR WAVE THEORY :    0.47366194968001946 m/s
SOULSBY APPROXIMATION:  0.29217128350136795 m/s
